In [ ]:
# Importing dependencies
import wikipedia
import pandas as pd
from random import sample 
import time
from math import log10,sqrt

list_searches_nba = ['Los Angeles Lakers', 'Golden State Warriors', 'Toronto Raptors',
                'Milwaukee Bucks', 'LA Clippers', 'Boston Celtics', 'Denver Nuggets',
                'Utah Jazz', 'Miami Heat', 'Houston Rockets', 'Dallas Mavericks',
                    'Atlanta Hawks', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls',
                    'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Memphis Grizzlies',
                    'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder']
list_searches_cl = ['Manchester City', 'Paris Saint Germain', 'Real Madrid', 'FC Barcelona',
                    'Liverpool', 'Juventus', 'Valancia', 'Atalanta Bergamo',
                    'FC Bayern Munchen', 'Atlético Madrid', 'SSC Napoli', 'Borussia Dortmund',
                    'Olympique Lyon', 'Tottenham Hotspur', 'RasenBallsport Leipzig', 'Ajax Amsterdam',
                    'Dinamo Zagreb', 'APOEL Nicosia', 'Dynamo kiev', 'Bayer 04 Leverkusen', 'Internazionale',
                    'FC Red Bull Salzburg', 'Galatasaray', 'Olympiacos', 'Hoffenhheim', 'Schalke 04',
                    'Feyenoord', 'PSV']

# Function for extracting the wikipedia pages of teams
def import_wikipedia(list_searches):
    data_searches = []
    for i in list_searches:
        p = wikipedia.page(i)
        data_searches.append(p.content)
    return data_searches
data_nba = import_wikipedia(list_searches_nba)
data_cl = import_wikipedia(list_searches_cl)
datasets = data_nba + data_cl

lst_search = list_searches_nba + list_searches_cl
stopwords= ['a','able','about','across','after','all','almost','also','am','among','an','and','any','are','as','at','be','because','been','but','by',
            'can','cannot','could','dear','did','do','does','either','else','ever','every','for','from','get','got','had','has','have','he','her','hers',
            'him','his','how','however','i','if','in','into','is','it','its','just','least','let','like','likely','may','me','might','most','must','my',
            'neither','no','nor','not','of','off','often','on','only','or','other','our','own','rather','said','say','says','she','should','since','so',
            'some','than','that','the','their','them','then','there','these','they','this','tis','to','too','twas','us','wants','was','we','were','what',
            'when','where','which','while','who','whom','why','will','with','would','yet','you','your']

In [ ]:
def mapper1(data, lst_search):
    final_output = []
    for i in range(len(data)):
        output_not_sorted = []
        filename = lst_search[i]
        # remove whitespace
        line = data[i].strip()
        # split into words
        words = line.split()
        for word in words:
            word=word.lower()
            if word not in stopwords:
                z=word+' * '+filename
                output = '%s \t %s' % (z, 1)
                output_not_sorted.append(output)
        sorted_output = sorted(output_not_sorted)
        final_output.append(sorted_output)

    return final_output

In [ ]:
# Computing the term occurences n
def reducer1(map_result):
    current_word = None
    current_count = 0
    word = None
    final_output = []

    for j in map_result:
        for line in j:
            line = line.strip()
            word, count = line.split('\t', 1)

            try:
                count = int(count)
            except ValueError:
                continue
            
            # The if-switch works because the output is mapped by key (here: word),
            # this is how it also works in Hadoop
            if current_word == word:
                current_count += count
            else:
                if current_word:
                    output = '%s \t %s' % (current_word, current_count)
                    final_output.append(output)
                current_count = count
                current_word = word

    # Outputting the last word if needed
    if current_word == word:
        output = '%s \t %s' % (current_word, current_count)
        final_output.append(output)
    return final_output

In [ ]:
%%time
map_result1 = mapper1(datasets, lst_search)
reduce_result1 = reducer1(map_result1)

In [ ]:
def mapper2(data_reducer):
    final_output = []
    for line in data_reducer:
        # remove whitespace
        line = line.strip()
        # splitting into words
        wordfilename,count=line.split('\t',1)
        wordfilename = wordfilename.strip()
        try:
            word,filename=wordfilename.split('*')
        except:
            pass
        z=word+' '+count;
        output = '%s \t %s' % (filename, z)
        final_output.append(output)
    return final_output

In [ ]:
# Computing the document lengths
def reducer2(map_result):
    current_word = None
    prev_filename = None
    current_count = 0
    word = None
    N=0
    df={}
    l1=[]
    final_output = []
    
    for line in map_result:
        line = line.strip()
        l1.append(line)
        filename,wordcount = line.split('\t', 1)
        wordcount = wordcount.strip()
        word,count = wordcount.split(' ', 1)
        count=int(count)
        if prev_filename == filename:
            N+=count
        else:
            if prev_filename != None:
                df[prev_filename]=N
            N=0
            prev_filename = filename
    df[prev_filename]=N

    for h in l1:
        filename,wordcount = h.split('\t', 1)
        wordcount = wordcount.strip()
        word,count = wordcount.split(' ', 1) 
        for k in df:
            if filename == k:
                wf=word+' '+filename
                nN=count+' '+str(df[k])
                output = '%s \t %s' % (wf,nN)
                final_output.append(output)
    return final_output

In [ ]:
%%time
map_result2 = mapper2(reduce_result1)
reduce_result2 = reducer2(map_result2)

In [ ]:
# Sorting the words alphabetically
def mapper3(reduce_result):
    output_list = []
    for line in reduce_result:
        line = line.strip()
        wf,nN=line.split('\t',1)
        w,f=wf.split(' ',1)
        z=f+' * '+nN+' '+str(1)
        output = '%s \t %s' % (w,z)
        output_list.append(output)
    final_output = sorted(output_list)
    return final_output

In [ ]:
# Computing the document-frequency of the terms
def reducer3(map_result):
    prev_word = None
    count = 1 
    word = None
    final_output = []
    for line in map_result:
        line = line.strip()
        w,z= line.split('\t', 1)
        w = w.strip()
        z = z.strip()
        f,nNc = z.split('*',1)
        f = f.strip()
        nNc = nNc.strip()
        n,Nc=nNc.split(' ',1)
        N,c=Nc.split(' ',1)
        if prev_word == w:
            count += int(c)
        else:
            if prev_word != None:
                output=w+' '+f+' \t '+n+' '+N+' '+str(count)
                final_output.append(output)
            count=1
            prev_word = w
    return final_output

In [ ]:
%%time
map_result3 = mapper3(reduce_result2)

In [ ]:
%%time
reduce_result3 = reducer3(map_result3)

In [ ]:
# Computing the TF-IDF weights
def mapper4(reduce_result):
    D=51.0
    final_output = []
    for line in reduce_result:
        # removing white space
        line = line.strip()
        # splitting the words
        wf,nNm=line.split('\t',1)
        wf = wf.strip()
        nNm = nNm.strip()
        n,N,m=nNm.split(' ',2)
        # transform to numeric for computation
        n=float(n)
        N=float(N)
        m=float(m)
        tfidf= (n/N)*log10(D/m)
        output = '%s \t %s' % (wf,tfidf)
        final_output.append(output)
    return final_output

In [ ]:
%%time
final = mapper4(reduce_result3)

In [ ]:
def compute_computation_times(data_nba, data_cl):
    running_times = []
    for i in range(5, 51, 5):
        new_data = [sample(data_nba,i), sample(data_cl,i)]
        data1, data2 = compute_frequency_matrix(new_data)
        start_time = time.time()
        igm1, igm2 = compute_igm(data1, data2)
        running_times.append(time.time() - start_time)
    return running_times

In [ ]:
# %%time
running_times = []
for i in range(5, 51, 5):
    new_data = sample(datasets,i)
    lst_searches = sample(lst_search, i)
    start_time = time.time()
    map_result1 = mapper1(new_data, lst_searches)
    reduce_result1 = reducer1(map_result1)
    map_result2 = mapper2(reduce_result1)
    reduce_result2 = reducer2(map_result2)
    map_result3 = mapper3(reduce_result2)
    reduce_result3 = reducer3(map_result3)
    final = mapper4(reduce_result3)
    running_times.append(time.time() - start_time)

In [ ]:
running_times

In [ ]:
final_output = []
final_tfidf = []
D=51.0
# input comes from STDIN (standard input)
for line in reduce_result3:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    wf,nNm=line.split('\t',1)
    wf = wf.strip()
    w,f = wf.split(' ',1)
    nNm = nNm.strip()
    n,N,m=nNm.split(' ',2)
    n=float(n)
    N=float(N)
    m=float(m)
    tfidf= (n/N)*log10(D/m)
    output = w
    final_output.append(output)
    final_tfidf.append(tfidf)

In [ ]:
data_tuples = list(zip(final_output,final_tfidf))
df1 = pd.DataFrame(data_tuples, columns=['word','tfidf'])
df1.sort_values('tfidf', ascending=False).head(15)

In [ ]:
# For plotting Figures
# Local vs Local
import matplotlib.pyplot as plt
import numpy as np

docs = [0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6, 4.0] # beginnen bij 0-
local_tfidf = [0.2649269104003906,
 0.3808310031890869,
 0.6321542263031006,
 0.887855052947998,
 1.214566946029663,
 1.4615440368652344,
 1.629607915878296,
 1.8721039295196533,
 1.9866352081298828,
 2.4448893070220947]
local_tfigm = [0.37888002395629883,
 0.8000800609588623,
 0.8628139495849609,
 1.406703233718872,
 1.6783900260925293,
 2.091007947921753,
 2.395679235458374,
 2.5690276622772217,
 3.1481950283050537,
 3.3638057708740234]

plt.plot(docs, local_tfidf, 'o-')
plt.plot(docs, local_tfigm, 'o-')
# plt.plot(x, 2 * x)
# plt.plot(x, 3 * x)
# plt.plot(x, 4 * x)

plt.title('Local TFIDF vs Local TFIGM', fontsize=18)
plt.legend(['y = TFIDF', 'y = TFIGM'], loc='upper left')
plt.xlabel('Size in Megabytes', fontsize=18)
plt.ylabel('Time (in seconds)', fontsize=16)

# plt.show()

In [ ]:
# Parallel vs Parallel
docs = [0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6, 4.0] # beginnen bij 0-
parallel_tfidf = [0.22279024124145508,
 0.41489458084106445,
 0.6862201690673828,
 0.7408955097198486,
 1.0512993335723877,
 1.1799609661102295,
 1.5287902355194092,
 1.686872959136963,
 1.8508782386779785,
 2.123838186264038]
parallel_tfigm = [0.4660379886627197,
 0.8262436389923096,
 0.8983533382415771,
 1.3413560390472412,
 1.4024183750152588,
 1.7298922538757324,
 2.121650457382202,
 2.3955447673797607,
 2.6349055767059326,
 3.0996642112731934]

plt.plot(docs, parallel_tfidf, 'o-')
plt.plot(docs, parallel_tfigm, 'o-')
# plt.plot(x, 2 * x)
# plt.plot(x, 3 * x)
# plt.plot(x, 4 * x)
plt.yticks([0.5, 1, 1.5, 2, 2.5, 3, 3.5])

plt.title('Parallel TFIDF vs Parallel TFIGM', fontsize=18)
plt.legend(['y = TFIDF', 'y = TFIGM'], loc='upper left')
plt.xlabel('Size in Megabytes', fontsize=18)
plt.ylabel('Time (in seconds)', fontsize=16)

# plt.show()

In [ ]:
# Local vs Parallel IGM
docs = [0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6, 4.0] # beginnen bij 0-
local_tfigm = [0.37888002395629883,
 0.8000800609588623,
 0.8628139495849609,
 1.406703233718872,
 1.6783900260925293,
 2.091007947921753,
 2.395679235458374,
 2.5690276622772217,
 3.1481950283050537,
 3.3638057708740234]
parallel_tfigm = [0.4660379886627197,
 0.8262436389923096,
 0.8983533382415771,
 1.3413560390472412,
 1.4024183750152588,
 1.7298922538757324,
 2.121650457382202,
 2.3955447673797607,
 2.6349055767059326,
 3.0996642112731934]

plt.plot(docs, local_tfigm, 'o-')
plt.plot(docs, parallel_tfigm, 'o-')
# plt.plot(x, 2 * x)
# plt.plot(x, 3 * x)
# plt.plot(x, 4 * x)

plt.title('Local TFIGM vs Parallel TFIGM', fontsize=18)
plt.legend(['y = Local', 'y = Parallel'], loc='upper left')
plt.xlabel('Size in Megabytes', fontsize=18)
plt.ylabel('Time (in seconds)', fontsize=16)

# plt.show()